In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [2]:
os.path.dirname(os.getcwd())

'/Users/antonioricciardi/projects/rl_relrepr_gymnasium'

In [3]:
# change working directory to the root of the project
os.chdir(os.path.dirname(os.getcwd()))

test_path = 'experiments/stitching_tests'

Preview data

In [22]:
env_id = "LunarLanderRGB" # "CarRacing-v2"
env_id_encoder = "LunarLanderRGB" # "CarRacing-v2"
env_info = "rgb"
mode = "absolute" # "absolute"
anc_alpha = "-1"

""" SHOW LOGGED DATA """
exp_path = f'{test_path}/{env_id}/{env_info}/{mode}/'
if mode == "relative":
    exp_path += f'a_{anc_alpha}/{env_id_encoder}_relu_stitching_results_ppo.csv'
else:
    exp_path += f'{env_id_encoder}_relu_stitching_results_ppo.csv'
# read stitching_results_relative_ppo with pandas
# df = pd.read_csv(f'{test_path}/{env_id}/{env_info}/{mode}/{env_id_encoder}_relu_stitching_results_ppo.csv', sep=',')
df = pd.read_csv(exp_path, sep=',')
df.tail()


,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time
635,6,red,red,4,4,LunarLanderRGB,LunarLanderRGB,-182.833740,147.0,ppo,0
636,7,red,red,4,4,LunarLanderRGB,LunarLanderRGB,272.074585,200.0,ppo,0
637,8,red,red,4,4,LunarLanderRGB,LunarLanderRGB,267.158569,213.0,ppo,0
638,9,red,red,4,4,LunarLanderRGB,LunarLanderRGB,-81.929680,178.0,ppo,0
639,10,red,red,4,4,LunarLanderRGB,LunarLanderRGB,249.494736,193.0,ppo,0


In [5]:
"""
file has the following columns:
track_seed,encoder_background,policy_background,score,max_score_reached,episode_length,algorithm
"""

'\nfile has the following columns:\ntrack_seed,encoder_background,policy_background,score,max_score_reached,episode_length,algorithm\n'

# CarRacing

In [24]:
# Read the CSV file into a DataFrame
env_id_encoder = "LunarLanderRGB" # "CarRacing-v2"
env_id = "LunarLanderRGB" # "CarRacing-v2"
# encoder_order = ['green', 'red', 'blue'] if env_id_encoder.startswith("CarRacing-v2") else ['plain', 'green', 'red']
encoder_order = ['green', 'red', 'blue'] if env_id_encoder.startswith("CarRacing-v2") else ['white', 'red']
if env_id.endswith("multicolor"):
    encoder_order = ['multicolor']
stitching_methods = ["absolute", "relative", "translate"]
stitching_method = stitching_methods[2]
anchoring_method = "random"
anchors_alpha = -1 # 0.999# -1

a_alpha = str(anchors_alpha)

model_activation = "relu"
model_algo = "ppo"


stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
if stitching_method == "relative":
    stitch_filename += f"a_{a_alpha}/"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

# stitch_filename = "experiments/stitching_tests/CarRacing-v2-multicolor/rgb/absolute/yellow-bg_CarRacing-v2_relu_stitching_results_ppo.csv"

print(stitch_filename, "\n")

df = pd.read_csv(stitch_filename)

# Convert 'encoder_background' to categorical with specified order
df['encoder_background'] = pd.Categorical(df['encoder_background'], categories=encoder_order, ordered=True)

# Convert 'policy_background' to categorical with specified order
df['policy_background'] = pd.Categorical(df['policy_background'], categories=encoder_order, ordered=True)

# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std']})
# show another column "score" next to max_score columnm containing the mean and std of the score
# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std'], 'score': ['mean', 'std']})

df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


experiments/stitching_tests/LunarLanderRGB/rgb/translate/random/LunarLanderRGB_relu_stitching_results_ppo.csv 



/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/988221905.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


score            
                                           mean         std
encoder_background policy_background                       
white              white              75.279707  190.072305
                   red                 1.842005  218.095778
red                white               2.271384  159.516153
                   red                50.912647  204.428507

In [25]:
# get encoder seed and policy seed that have top average score for encoder_background=green and policy_background=red
df[(df['encoder_background'] == 'green') & (df['policy_background'] == 'red')].sort_values(by='score', ascending=False).head(1)

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time


In [26]:
df[(df['encoder_background'] == 'green') & (df['policy_background'] == 'red')].sort_values(by='score', ascending=True).head(1)

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time


In [27]:
# print the total number of rows
print(f"Total number of rows: {df.shape[0]}")

Total number of rows: 640


In [28]:
# show the mean score of all the experiments where encoder_background == policy_background
# compute the mean score
print('Same encoder and policy background')
print(df[df['encoder_background'] == df['policy_background']]['score'].mean())
print(df[df['encoder_background'] == df['policy_background']]['score'].std())
print()
# show the mean score of all the experiments, excluding where encoder_background == policy_background
# compute the mean score
print('Different encoder and policy background')
print(df[df['encoder_background'] != df['policy_background']]['score'].mean())
print(df[df['encoder_background'] != df['policy_background']]['score'].std())

print('Cumulative scores')
print(df['score'].mean())
print(df['score'].std())

# get scores for when encoder_seed == policy_seed and encoder_background == policy_background
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].std())

# get all the rows where encoder_seed == policy_seed and encoder_background == policy_background
equal_seed = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]
# compute aggregate score over environment seeds, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
equal_seed_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
# rename mean score column to mean_score
equal_seed_scores.columns = ['mean_score', 'std_score']
# aggregate encoder_seed and policy_seed into one column called seed
equal_seed_scores['seed'] = equal_seed_scores.index.get_level_values('encoder_seed').astype(str)
# disaggregate encoder_seed and policy_seed, preserve encoder_background and policy_background
equal_seed_scores.index = equal_seed_scores.index.droplevel(['encoder_seed', 'policy_seed'])
# compute mean and std over seeds

equal_seed_scores

Same encoder and policy background
63.09617663621903
197.44878364697416

Different encoder and policy background
2.056694734096527
190.76476197614787
Cumulative scores
32.576435685157776
196.3734705687735
Same encoder and policy seed
225.47769923210143
95.69399397288444


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/2909045596.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  equal_seed_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()


,,mean_score,std_score,seed
encoder_background,policy_background,,,
white,white,240.867443,89.657443,1
red,red,221.007354,94.387902,1
white,white,266.544397,18.947405,2
red,red,247.429843,81.370017,2
white,white,266.223134,20.430097,3
red,red,152.809093,131.016874,3
white,white,201.229495,138.217637,4
red,red,207.710835,91.409636,4


When encoder seed and policy seed are different, or when the encoder and policy background are different

In [29]:
""" STITCHING """
# get all the rows where encoder_seed != policy_seed or encoder_background != policy_background
equal_seed = df[(df['encoder_seed'] != df['policy_seed']) | (df['encoder_background'] != df['policy_background'])]
# compute aggregate score over environment seeds, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
diff_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
# rename mean score column to mean_score
diff_scores.columns = ['mean_score', 'std_score']
# aggregate encoder_seed and policy_seed into one column called seed
diff_scores['seed'] = diff_scores.index.get_level_values('encoder_seed').astype(str)
# disaggregate encoder_seed and policy_seed, preserve encoder_background and policy_background
diff_scores.index = diff_scores.index.droplevel(['encoder_seed', 'policy_seed'])
# compute mean and std over seeds

diff_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': ['mean', 'std']})


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/749558234.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  diff_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/749558234.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  diff_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': ['mean', 'std']})


mean_score           
                                           mean        std
encoder_background policy_background                      
white              white              19.134237  67.921435
                   red                 1.842005  85.517270
red                white               2.271384  72.624214
                   red                -1.196232  85.012270

Compute mean scores over different seeds for when encoder_background and policy_background are the same, and the seed too

In [30]:
""" NO STITCHING """
equal_background_scores = equal_seed_scores[equal_seed_scores.index.get_level_values('encoder_background') == equal_seed_scores.index.get_level_values('policy_background')]
average_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'mean', 'std_score': 'std'})
average_scores

/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/703665784.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'mean', 'std_score': 'std'})


mean_score  std_score
encoder_background policy_background                       
white              white              243.716117  57.916548
                   red                       NaN        NaN
red                white                     NaN        NaN
                   red                207.239281  21.707055

In [31]:
median_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'median', 'std_score': 'std'})
median_scores


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/790856185.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'median', 'std_score': 'std'})


mean_score  std_score
encoder_background policy_background                       
white              white              253.545288  57.916548
                   red                       NaN        NaN
red                white                     NaN        NaN
                   red                214.359095  21.707055

Get maximum scores

In [32]:
# save idxmax to a list
idxmax_list = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])].groupby(['encoder_background']).agg({'score': ['max', 'mean', 'std', 'idxmax']})['score']['idxmax'].tolist()
print(idxmax_list)
# select encoder_background, policy_background, seeds from the list
selection = df.iloc[idxmax_list][['encoder_background', 'policy_background', 'encoder_seed', 'policy_seed', 'score']]
selection

[58, 538]


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/128057864.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idxmax_list = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])].groupby(['encoder_background']).agg({'score': ['max', 'mean', 'std', 'idxmax']})['score']['idxmax'].tolist()


,encoder_background,policy_background,encoder_seed,policy_seed,score
58,white,white,2,2,306.233582
538,red,red,2,2,319.991150


Get minimum scores

In [33]:
# get minimum score
min_score = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].min()
# get the row with the minimum score
min_score_row = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background']) & (df['score'] == min_score)]
min_score_row

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time
154,5,white,white,4,4,LunarLanderRGB,LunarLanderRGB,-59.385349,100.0,ppo,0.031


In [34]:

# get maximum score
max_score = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].max()
# get the row with the maximum score
max_score_row = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background']) & (df['score'] == max_score)]
max_score_row

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time
538,9,red,red,2,2,LunarLanderRGB,LunarLanderRGB,319.99115,202.0,ppo,0.033


# CarRacing multicolor

In [35]:
# Read the CSV file into a DataFrame
background = "multicolor"
env_id_encoder = "CarRacing-v2-multicolor"
env_id = "CarRacing-v2"
encoder_order = ['multicolor'] if env_id_encoder.startswith("CarRacing-v2") else ['plain', 'green', 'red']
controller_order = ['green', 'red', 'blue'] if env_id_encoder.startswith("CarRacing-v2") else ['plain', 'green', 'red']
stitching_methods = ["absolute", "relative", "translate"]
stitching_method = stitching_methods[0]
anchoring_method = "fps"
anchors_alpha = 0.999

a_alpha = str(anchors_alpha)

model_activation = "relu"
model_algo = "ppo"


stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
if stitching_method == "relative":
    stitch_filename += f"a_{a_alpha}/"
# stitch_filename += f"{background}-bg_{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

df = pd.read_csv(stitch_filename)

# Convert 'encoder_background' to categorical with specified order
df['encoder_background'] = pd.Categorical(df['encoder_background'], categories=encoder_order, ordered=True)

# Convert 'policy_background' to categorical with specified order
df['policy_background'] = pd.Categorical(df['policy_background'], categories=controller_order, ordered=True)

# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std']})
# show another column "score" next to max_score columnm containing the mean and std of the score
# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std'], 'score': ['mean', 'std']})

df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/3966089445.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


score            
                                           mean         std
encoder_background policy_background                       
multicolor         green              16.355740  133.792033
                   red               -26.434182   67.166386
                   blue              -55.962946   47.546080

In [36]:
# print results when encoder_seed and policy_seed are the same
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed'])]['score'].std())

# print results when encoder_seed and policy_seed are different
print('Different encoder and policy seed')
print(df[(df['encoder_seed'] != df['policy_seed'])]['score'].mean())
print(df[(df['encoder_seed'] != df['policy_seed'])]['score'].std())

# print cumulative results
print('Cumulative scores')
print(df['score'].mean())
print(df['score'].std())

Same encoder and policy seed
20.134520575404167
155.02669634141373
Different encoder and policy seed
-36.06323501467705
57.95641462205124
Cumulative scores
-22.013796117156744
94.99484788998933


# ATARI

In [37]:
# Read the CSV file into a DataFrame
env_id = "BoxingNoFrameskip-v4"
env_id_encoder = "BoxingNoFrameskip-v4"
encoder_order = ['plain', 'green', 'red']
stitching_methods = ["absolute", "relative", "translate"]
stitching_method = stitching_methods[2]
anchoring_method = "fps"

anchors_alpha = 0.999

a_alpha = str(anchors_alpha)

model_activation = "relu"
model_algo = "ppo"

stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
if stitching_method == "relative":
    stitch_filename += f"a_{a_alpha}/"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

df = pd.read_csv(stitch_filename)

# Convert 'encoder_background' to categorical with specified order
df['encoder_background'] = pd.Categorical(df['encoder_background'], categories=encoder_order, ordered=True)

# Convert 'policy_background' to categorical with specified order
df['policy_background'] = pd.Categorical(df['policy_background'], categories=encoder_order, ordered=True)

# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std']})
# show another column "score" next to max_score columnm containing the mean and std of the score
# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std'], 'score': ['mean', 'std']})

df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/908304895.py:35: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


score           
                                         mean        std
encoder_background policy_background                    
plain              plain              90.3125   5.287958
                   green              49.1875  42.379584
                   red                46.3125  53.231217
green              plain              86.4375   5.691734
                   green              59.1250  47.824506
                   red                55.8750  57.821420
red                plain              80.4375  22.196002
                   green              51.5625  31.956155
                   red                52.6250  51.771131

In [38]:
# print the dataframe where encoder_background == policy_background and encoder_background == 'plain'. Show the score, encoder_seed and policy_seed
df[(df['policy_background'] == 'plain') & (df['encoder_background'] == 'plain')][['score', 'encoder_background', 'policy_background', 'encoder_seed', 'policy_seed']]

,score,encoder_background,policy_background,encoder_seed,policy_seed
0,94.0,plain,plain,0,0
1,93.0,plain,plain,0,1
2,94.0,plain,plain,0,2
3,86.0,plain,plain,0,3
4,85.0,plain,plain,1,0
5,96.0,plain,plain,1,1
6,86.0,plain,plain,1,2
7,93.0,plain,plain,1,3
8,88.0,plain,plain,2,0
9,84.0,plain,plain,2,1


In [39]:
# print the total number of rows
print(f"Total number of rows: {df.shape[0]}")

# get scores for when encoder_seed == policy_seed and encoder_background == policy_background
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].std())

# show the mean score of all the experiments where encoder_background == policy_background
# compute the mean score
print('Same encoder and policy background')
print(df[df['encoder_background'] == df['policy_background']]['score'].mean())
print(df[df['encoder_background'] == df['policy_background']]['score'].std())
print()
# show the mean score of all the experiments, excluding where encoder_background == policy_background
# compute the mean score
print('Different encoder and policy background')
print(df[df['encoder_background'] != df['policy_background']]['score'].mean())
print(df[df['encoder_background'] != df['policy_background']]['score'].std())

print('Cumulative scores')
print(df['score'].mean())
print(df['score'].std())

# get scores for when encoder_seed == policy_seed and encoder_background == policy_background
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].std())

# get all the rows where encoder_seed == policy_seed and encoder_background == policy_background
equal_seed = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]
# compute mean score over colors, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
equal_seed_scores = equal_seed.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
equal_seed_scores

Total number of rows: 144
Same encoder and policy seed
95.58333333333333
2.6443192398846707
Same encoder and policy background
67.35416666666667
43.250625174836316

Different encoder and policy background
61.635416666666664
41.88659370796509
Cumulative scores
63.541666666666664
42.28125524927737
Same encoder and policy seed
95.58333333333333
2.6443192398846707


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/1182102584.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  equal_seed_scores = equal_seed.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()


score          
                                       mean       std
encoder_background policy_background                 
plain              plain              94.25  1.258306
green              green              96.25  2.986079
red                red                96.25  3.403430

When encoder seed and policy seed are different, or when the encoder and policy background are different (in which case we do not care whether the seed is the same or not)

In [40]:
# get all the rows where encoder_seed != policy_seed or encoder_background != policy_background
diff = df[(df['encoder_seed'] != df['policy_seed']) | (df['encoder_background'] != df['policy_background'])]
# compute aggregate score over environment seeds, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
diff_scores = diff.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
diff_scores


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_62051/1573178490.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  diff_scores = diff.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()


score           
                                           mean        std
encoder_background policy_background                      
plain              plain              89.000000   5.493798
                   green              49.187500  42.379584
                   red                46.312500  53.231217
green              plain              86.437500   5.691734
                   green              46.750000  49.479335
                   red                55.875000  57.821420
red                plain              80.437500  22.196002
                   green              51.562500  31.956155
                   red                38.083333  52.239759

# ANALYSIS

In [41]:
env_id = "CarRacing-v2"
env_info = "rgb"
anchoring_method = "fps"

""" SHOW LOGGED DATA """

# read stitching_results_relative_ppo with pandas
df1 = pd.read_csv(f'{test_path}/{env_id}/{env_info}/absolute/relu_stitching_results_ppo.csv', sep=',')
df1.head()
stitch_filename_2 = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename_2 += f"{anchoring_method}/"
stitch_filename_2 += f"{model_activation}_stitching_results_{model_algo}.csv"
# df2 = pd.read_csv(f'{test_path}/{env_id}/{env_info}/translate/relu_stitching_results_ppo.csv', sep=',')
df2 = pd.read_csv(stitch_filename_2)



# join the two dataframes adding a new column called "method". Use "absolute" under method for df1 and "translate" for df2
df1['method'] = 'absolute'
# sum 40 to df1 env_seed column
df1['env_seed'] = df1['env_seed'] + 39
df2['method'] = 'translate'

# create df3 containing data where encoder_background and policy_background are the same, and remove it from df1 and df2
df3 = df1[df1['encoder_background'] == df1['policy_background']]
df3['method'] = 'absolute_original'
df1 = df1[df1['encoder_background'] != df1['policy_background']]
df4 = df2[df2['encoder_background'] == df2['policy_background']]
df4['method'] = 'translate_original'
df2 = df2[df2['encoder_background'] != df2['policy_background']]

# concatenate the four dataframes
df = pd.concat([df1, df2, df3, df4])
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'experiments/stitching_tests/CarRacing-v2/rgb/absolute/relu_stitching_results_ppo.csv'

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

""" df contains the following columns: env_seed	encoder_background	policy_background	score	max_score_reached	episode_length	algorithm
Create a histogram with x axis being the env_seed and y axis being the score. Use "absolute" and "translation" as colors.
Each score must be a bar.
"""
fig = px.box(df, x="env_seed", y="score", color="method", facet_row="encoder_background", height=800)
#fig = px.histogram(df, x="env_seed", y="score", color="method", barmode="group", histfunc="avg")
fig.show()


In [ ]:
Allora puoi fare un istrogramma con plotly express. Prendi in input il df e metti come x il track_seed,
come y il valore di score raggiunto o quello che è, come color il metodo usato (che sarà o absolute o translation)